# Cliente

Função usada para converter a mensagem em bit.

In [ ]:
import socket
import binascii

from convertCRC import CRC
from quadro import QuadroDados

def transformaEmBit(listaBytes):
    resultado = "0b"
    for item in listaBytes:
        item = bin(int(str(item.hex()), 16))[2:]
        while(len(item) < 8):
            item = "0" + item
        resultado += item
    return resultado

Dividimos a mensangem em tamanhos permitidos a serem enviados pela conexão

In [ ]:
def divideMsg(msg, tamMax):
    listMsg = []
    pos = 0
    tamList = len(msg)
    while(pos < tamList):
        if((pos + tamMax) < tamList):
            listMsg.append(msg[pos : pos + tamMax])
        else:
            listMsg.append(msg[pos:])
        pos += tamMax

    return listMsg

Aqui no reenvio tratamos como um envio normal, ou seja, tudo aquilo que é feito no envio, fazemos de novo.


In [ ]:
def reenviaQuadro(conn, quadro, bitSequence):
    enviado = False
    while enviado:
        conn.send(quadro)
        # Capturando o quadro de confirmaçao que é enviado pela conexao
        quadConfirmacao = b''
        # capturando 'bit delimiter'
        quadConfirmacao += conn.recv(1)
        # Caso esteja vazio, quer dizer que quadro se perdeu no caminho
        if(len(quadConfirmacao) <= 0):
            enviado = False
            continue

        # Capturando 'bit sequence'
        quadConfirmacao += conn.recv(1)
        bitSequence = quadConfirmacao[1]
        for i in range(8):
            quadConfirmacao += conn.recv(1)

        bitSequenceRequest = ['0', '1'] [bitSequence >= 128]
        # Teste se resquest é referente ao ultimo quadro enviado pelo cliente

        if(bitSequenceRequest == bitSequence):
            # Caso seja, eu vejo se mensagem foi recebida com sucesso
            if(bitSequence % 2 == 0): # sucesso é: bit de sequencia ter 1 no ACK
                enviado = False
                continue
            else:
                enviado = True
                continue
        else:
            enviado = False
            continue

Declaramos aqui:
- Host: Enderenço IP do Servidor
- Port: Porta onde esta o servidor
- Max_LENGHT: Tamanho maximo para a mensagem a ser enviada

In [ ]:
    HOST = "127.0.0.1"     
    PORT = 6060
    MAX_LENGHT = 255

Criamos aqui o Socket de conexão e capturamos o IP para iniciarmos uma conexão logo abaixo declaramos a mensagem padrão para o envio e dividimos ela para o tamanho maximo.

In [ ]:
    conn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    dest = (HOST, PORT)
    conn.connect(dest)

    meuIP = conn.getsockname()[0]

    msg  = "Todos nos ja ouvimos falar certamente em redes de comunicacao (tambem designadas de redes informaticas ou redes de dados).\n"
    msg += "Uma \"rede\" (na Area da informatica), e definida como um conjunto de equipamentos interligados entre si, e que permitem o transporte"
    msg = divideMsg(msg, MAX_LENGHT)
    bitSequenceEnvio = ""


Primeiro criamos um quadro de envio da mensagem e colocamos na posição de "msg", então capturamos o quadro de confirmação que é enviado pela conexão, depois capturamos o "bit delimiter" e testamos caso esteja vazio, quer dizer que o quadro se perdeu no caminho.
Depois capturamos o "bit sequence" e testamos se o requeste é referente ao ultimo quadro enviado pelo cliente, caso seja testamos se a mensagem foi enviada com sucesso.
Para finalizar enviamos um pacote de desconexão.

In [ ]:
for indice,mensagem in enumerate(msg):
        bitSequenceEnvio = str(indice%2)

        msg[indice] = QuadroDados(HOST, meuIP, mensagem, bitSequenceEnvio).getQuadro()
        
        conn.send(msg[indice])

        quadConfirmacao = b''
        
        quadConfirmacao += conn.recv(1)
        
        if(len(quadConfirmacao) <= 0):
            reenviaQuadro(conn, msg[indice], bitSequenceEnvio)
            continue

        quadConfirmacao += conn.recv(1)
        bitSequence = quadConfirmacao[1]
        for i in range(8):
            quadConfirmacao += conn.recv(1)
        
        bitSequenceRequest = ['0', '1'] [bitSequence >= 128]
        
        if(bitSequenceRequest == bitSequenceEnvio):
            
            if(bitSequence % 2 == 0):
                print("Quadro corrompido")
            else:
                print("Quadro enviado ao servidor com sucesso")
        else:
            reenviaQuadro(conn, msg[indice], bitSequenceEnvio)
            continue
            
        bitSequenceEnvio = ['1', '0'][bitSequenceEnvio == '1']
        msgDesconexao = QuadroDados(HOST, meuIP, "#desconectar#", bitSequenceEnvio).getQuadro()
        conn.send(msgDesconexao)

        conn.close()
    


<div align='center'>
   <a href='servidor.ipynb' style="padding-right: 2em;"><i class="fa fa-angle-double-left"></i> Servidor</a>
   <a href='calculo_CRC.ipynb'>Cálculo CRC<i class="fa fa-angle-double-right"></i></a>
</div>